# Movielens Recommendation System

by Umur Türkay and Yasemin Alpay

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import scipy.linalg as la
import pandas as pd
import heapq as hp
import math


First we read the rating data using pandas library.

In [2]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp'] 
mnames  = ['movie_id', 'title', 'genres']
ratings = pd.read_table('data/ratings.dat', sep='::', header=None, names=rnames, engine='python')
movies  = pd.read_table(u'data/movies.dat', sep='::', header=None, names=mnames, engine = 'python')

In [3]:
movies [:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [5]:
allratings = pd.merge(movies, ratings)
allratings[:10]

,movie_id,title,genres,user_id,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474
5,1,Toy Story (1995),Animation|Children's|Comedy,18,4,978154768
6,1,Toy Story (1995),Animation|Children's|Comedy,19,5,978555994
7,1,Toy Story (1995),Animation|Children's|Comedy,21,3,978139347
8,1,Toy Story (1995),Animation|Children's|Comedy,23,4,978463614
9,1,Toy Story (1995),Animation|Children's|Comedy,26,3,978130703


## Algorithm 1 -Item Based k-NN

We used collaborative filtering and implemented item based k-NN algorithm to determine the recommendations. In order to implement the k-NN algorithm, we had to define a similarity function. 

The similarity functions that can be used in k-NN can vary, like Euclidean, Hamming, or Cosine similarity. We have chosen adjusted cosine similarity for our implementation. (Also can be found in: "Recommendation System Based on Collaborative Filtering", Chapter 3.1, by Zheng Wen, December 12, 2008).

Algorithm follows:

1-Each movie is passed to the similarity function. Similarity function need users and these users are selected from a set which the user u1 and user u2 voted for the movies that similarity function finds the similarity for.

2-Most similar k movies for each movie is found using the heap structure. 

3-We calculate weighted average to predict the rating that the user u gives to movie m. As the weight, we use the similarity values of the movie m and the most k similar movies to m
correspondingly.




Similarity function:

In [6]:
def sim(m1, m2):  #Movies to be compared 
    sum = 0
    diffSquare1 = 0
    diffSquare2 = 0
    M1 = A[A[:,1] == m1]
    M2 = A[A[:,1] == m2]
    commonUsers = np.intersect1d(M1[:,0], M2[:,0])
    for i in range(len(commonUsers)):
        userRatingMovie1 = np.array(ratings[(ratings.user_id == commonUsers[i]) & (ratings.movie_id == m1)]['rating'])[0]
        userRatingMovie2 = np.array(ratings[(ratings.user_id == commonUsers[i]) & (ratings.movie_id == m2)]['rating'])[0]
        U = A[A[:,0] == commonUsers[i]]
        userAvgRating = float(np.mean(U[:,2]))
        
        diffMov1 = float(userRatingMovie1 - userAvgRating)
        diffMov2 = float(userRatingMovie2 - userAvgRating)
        
        sum = float(sum + (diffMov1)*(diffMov2))
        
        diffSquare1 = diffSquare1 + pow(diffMov1,2)
        diffSquare2 = diffSquare2 + pow(diffMov2,2)
        
    sim = float(float(sum)/float(math.sqrt(diffSquare1*diffSquare2)))
    return sim

Heap:

In [7]:
# Use a heap to store the smallest items
# Define an object and overload custom comparison operators
class tup:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
                
    def __lt__(self, other):
        '''Redefine for max-heap'''
        return self.val > other.val
    
    def __le__(self, other):
        return self.val <= other.val
 
    def __eq__(self, other):
        return self.val == other.val
    
    def __ne__(self, other):
        return self.val != other.val

    def __gt__(self, other):
        return self.val > other.val

    def __ge__(self, other):
        return self.val >= other.val

    def __str__(self):
        return '{:.3},{:d}'.format(self.val,self.idx)

In [8]:
user_id = 1 #user id to give recommendation
heapSize

usersRatedMovies= np.array(ratings[(ratings.user_id == user_id)]['movie_id'])
allMovies = np.array(ratings['movie_id'])
usersUnRatedMovies = list(set(allMovies) - set(usersRatedMovies))

N = allMovies.shape[0]   
comparedMovie = 1
rankings = []
for k in range(len(usersUnRatedMovies)):
    if(len(usersUnRatedMovies)>k):
        comparedMovie = usersUnRatedMovies[k]
        simSum = 0
        for m in range(N):
            simValue = sim(comparedMovie, allMovies[m])
            simSum += simValue
            tp = tup(simValue, allMovies[m])
            if tp <= heap[0]:
                hp.heapreplace(heap, tp)
        prediction = 0.0
        for j in range(len(heap)):
            h = hp.heappop(heap)
            prediction += h.val*ratings[(ratings.user_id == 1)][(ratings.movie_id == h.idx)]['rating']
        prediction = prediction/simSum
        rankings.append(prediction, comparedMovie)
        
rankings.sort()
rankings.reverse()


11
****
1000209
661


## Algorithm 2 - Stochastic Gradient Descent

We implemented SGD as our second algorithm.

In [8]:
userM = np.matrix(ratings.user_id).T
movieM = np.matrix(ratings.movie_id).T
ratingM = np.matrix(ratings.rating).T

userLength = np.hstack(set(ratings['user_id']))[-1]
movieLength = np.hstack(set(ratings['movie_id']))[-1]

userStart = np.matrix([0]*(userLength+1)).T 
movieStart = np.matrix([0]*(movieLength+1)) 
Y = userStart*movieStart
M = Y
Ys = np.hstack([ratingM,userM,movieM])
Ysize = Ys.shape[0]

We have Y as our sparse matrix here. Y matrix has users as its rows and movies as its columns. Elements are ratings.
On the other hand, Ys matrix is a combination of ratings, users and movies.

In [9]:
Ys[:10]

matrix([[   5,    1, 1193],
        [   3,    1,  661],
        [   3,    1,  914],
        [   4,    1, 3408],
        [   5,    1, 2355],
        [   3,    1, 1197],
        [   5,    1, 1287],
        [   5,    1, 2804],
        [   4,    1,  594],
        [   4,    1,  919]], dtype=int64)

SGD Algorithm:
    
1-First, we create random A and B matrices. 

2-Then we find the error by substracting A*B from our actual Y matrix.

3-We find the elements of A and B iteratively by reducing the learning rate(eta).

4-This way we fill the blanks of sparse Y matrix and now have the movie predictions.

5-Any prediction/recommendation can be made by this final matrix.


In [ ]:
A = np.mat(np.random.rand(userLength+1, 1))
B = np.mat(np.random.rand(1, movieLength+1))

EPOCH = 5
Eta = 0.1
eta = Eta

for i in range(EPOCH):
    E = np.array(M)*np.array(Y - A*B)
    Err = np.sum(E*E)/np.sum(np.array(M))
        
    for k in range(Ysize):
        u = Ys[k,1]
        m = Ys[k,2]
        
        err = Ys[k,0] - A[u,:]*B[:,m]
        
        temp_A = A[u,:] + eta*err[0,0]*B[:,m].T
        B[:,m]   = B[:,m] + eta*err[0,0]*A[u,:].T
        A[u,:]   = temp_A
    
    eta = Eta*1./(i+1)

In [ ]:
res = A*B #final result matrix
user_id = 5 #example user id to recommend
userResult = [(0,0)]
for m in range (movieLength):
    movieTuple = (res[user_id,m],m)
    userResult.append(movieTuple)
userResult = sorted(userResult, key=lambda tup: tup[0], reverse=True)

#10 movies to recommend
for n in range(10):
    movie_id = userResult[n][1]
    movie_name = allratings[(allratings.movie_id == movie_id)]['title'][0]
    print(movie_id, movie_name)
